In [1]:
!pip install -q --upgrade pip
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.o

In [2]:
!pip uninstall jaxlib
!pip install -U jaxlib==0.4.2+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Found existing installation: jaxlib 0.3.25+cuda11.cudnn805
Uninstalling jaxlib-0.3.25+cuda11.cudnn805:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/jaxlib-0.3.25+cuda11.cudnn805.dist-info/*
    /usr/local/lib/python3.8/dist-packages/jaxlib/*
Proceed (Y/n)? Y
  Successfully uninstalled jaxlib-0.3.25+cuda11.cudnn805
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 MB 7.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

#/content/drive/MyDrive/MasterarbeitData

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [11]:
!git config --global user.email "maxhahn92@outlook.de"
!git config --global user.name "Max Hahn"

In [14]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 429 bytes | 9.00 KiB/s, done.
From https://github.com/maxamas/two_WL_NTK
   61ffe52..0af2eaa  main       -> origin/main
Updating 61ffe52..0af2eaa
Fast-forward
 utils.py | 6 ++++++
 1 file changed, 6 insertions(+)


In [20]:
from utils import *

In [6]:
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [15]:
# preproces data
dataset = TUDataset(root="Masterarbeit", name="MUTAG")

# make lists of dense adjacency matrix, node feature array and y
graps_node_features = list()
As = list()
ys = list()

for data in dataset:
  As.append(to_dense(data.edge_index, len(data.x)))
  graps_node_features.append(np.array(data.x))
  ys.append(np.array(data.y))


# get the initial edge feature matrix
# when the graph specification has only node features
# as a batch x nodes x nodes x channels array

nb_graphs = len(graps_node_features)
max_nodes = len(max(graps_node_features, key=lambda x: len(x)))
feature_dim = graps_node_features[0][0].shape[0]

graphs_edge_features = np.zeros((nb_graphs, max_nodes, max_nodes, feature_dim))

for k, node_features in enumerate(graps_node_features):
    for i, node_feature in enumerate(node_features):
        graphs_edge_features = graphs_edge_features.at[k,i,i,:].set(node_feature)

# calculate the pattern array for a edge feature matrix
# of shape batch x nodes x nodes x channels array

As = [zero_append(a, (max_nodes, max_nodes)) for a in As]
As = np.array(As)

As_3 = r_power_adjacency_matrix(As, 3)
As_3_int = neigbourhood_intersections(As_3)
As_3_pattern = np.transpose(np.array(np.nonzero(As_3_int)))

print(As_3_pattern.shape)
print(graphs_edge_features.shape)

Extracting Masterarbeit/MUTAG/MUTAG.zip
Processing...
Done!


(353051, 4)
(188, 28, 28, 7)


In [16]:
import os
newpath = "/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [17]:
# save preprocessed data

np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/ys", ys)
np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/As_3_pattern", As_3_pattern)
np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/graphs_edge_features", graphs_edge_features)


In [ ]:
# load preprocessed data

ys = np.load(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/ys.npy")
As_3_pattern = np.load(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/As_3_pattern.npy")
graphs_edge_features = np.load(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/graphs_edge_features.npy")

print(As_3_pattern.shape)
print(graphs_edge_features.shape)

(353051, 4)
(188, 28, 28, 7)


In [18]:
import shutil
shutil.rmtree("/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG")

In [23]:
# split the preprocessed data into pieces, to save memory

import os
newpath = "/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

size = 4
for i in range(47):
  for j in range(47):
    x1_batch_range = list(range(i*size,(1+i)*size))
    x2_batch_range = list(range(j*size,(1+j)*size))

    x1 = graphs_edge_features[x1_batch_range,:]
    x2 = graphs_edge_features[x2_batch_range,:]

    p1 = As_3_pattern[column_in_values(As_3_pattern[:,0], np.array(x1_batch_range)),:]
    p2 = As_3_pattern[column_in_values(As_3_pattern[:,0], np.array(x2_batch_range)),:]  

    np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/x1_{i*size}_{(1+i)*size}", x1)
    np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/x2_{j*size}_{(1+j)*size}", x2)
    np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/p1_{i*size}_{(1+i)*size}", p1)
    np.save(f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG/p2_{j*size}_{(1+j)*size}", p2)

In [29]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   preprocessing.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Masterarbeit/
	two-wl-ntk-kernel-calcualtion.ipynb



In [30]:
!git add preprocessing.ipynb

In [26]:
!git commit -m "finish preprocessing"

[main 2017718] finish preprocessing
 1 file changed, 1 insertion(+), 1 deletion(-)
